In [ ]:
pip install -q pyyaml h5py #Required to save models in HDF5 format

In [13]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
#print(tf.version.VERSION)

In [14]:
ufc = pd.read_csv("data.csv")
ufc.head()

BPrev  BStreak  B_Age  B_Height                          B_HomeTown  B_ID  \
0      1        1   23.0     182.0                        Trento Italy  2783   
1      0        0   32.0     175.0  Careiro da Várzea, Amazonas Brazil  2208   
2      2        0   38.0     172.0                      Kanagawa Japan   721   
3      0        0   23.0     170.0                      Tijuana Mexico  2825   
4      3        1   30.0     167.0                     Spokane, WA USA  2260   

          B_Location                 B_Name  B_Weight  \
0  Mezzocorona Italy         Marvin Vettori        84   
1   Pharr, Texas USA  Carlos Diego Ferreira        70   
2        Tokyo Japan          Takanori Gomi        70   
3     Tijuana Mexico         Brandon Moreno        56   
4    Spokane, WA USA     Elizabeth Phillips        61   

   B__Round1_Grappling_Reversals_Landed  ...  R__Round5_TIP_Ground Time  \
0                                   0.0  ...                        NaN   
1                                   NaN  ...                        NaN   
2                                   0.0  ...                        NaN   
3                                   NaN  ...                        NaN   
4                                   0.0  ...                        NaN   

   R__Round5_TIP_Guard Control Time  R__Round5_TIP_Half Guard Control Time  \
0                               NaN                                    NaN   
1                               NaN                                    NaN   
2                               NaN                                    NaN   
3                               NaN                                    NaN   
4                               NaN                                    NaN   

   R__Round5_TIP_Misc. Ground Control Time  R__Round5_TIP_Mount Control Time  \
0                                      NaN                               NaN   
1                                      NaN                               NaN   
2                                      NaN                               NaN   
3                                      NaN                               NaN   
4                                      NaN                               NaN   

   R__Round5_TIP_Neutral Time  R__Round5_TIP_Side Control Time  \
0                         NaN                              NaN   
1                         NaN                              NaN   
2                         NaN                              NaN   
3                         NaN                              NaN   
4                         NaN                              NaN   

   R__Round5_TIP_Standing Time   winby  winner  
0                          NaN     DEC     red  
1                          NaN     SUB    blue  
2                          NaN  KO/TKO     red  
3                          NaN     SUB    blue  
4                          NaN     DEC     red  

[5 rows x 895 columns]

In [15]:
ufc = ufc[ufc.winner != 'no contest']
ufc = ufc[ufc.winner != 'draw']
ufc_zero = ufc.fillna(0)
ufc_zero.head()

BPrev  BStreak  B_Age  B_Height                          B_HomeTown  B_ID  \
0      1        1   23.0     182.0                        Trento Italy  2783   
1      0        0   32.0     175.0  Careiro da Várzea, Amazonas Brazil  2208   
2      2        0   38.0     172.0                      Kanagawa Japan   721   
3      0        0   23.0     170.0                      Tijuana Mexico  2825   
4      3        1   30.0     167.0                     Spokane, WA USA  2260   

          B_Location                 B_Name  B_Weight  \
0  Mezzocorona Italy         Marvin Vettori        84   
1   Pharr, Texas USA  Carlos Diego Ferreira        70   
2        Tokyo Japan          Takanori Gomi        70   
3     Tijuana Mexico         Brandon Moreno        56   
4    Spokane, WA USA     Elizabeth Phillips        61   

   B__Round1_Grappling_Reversals_Landed  ...  R__Round5_TIP_Ground Time  \
0                                   0.0  ...                        0.0   
1                                   0.0  ...                        0.0   
2                                   0.0  ...                        0.0   
3                                   0.0  ...                        0.0   
4                                   0.0  ...                        0.0   

   R__Round5_TIP_Guard Control Time  R__Round5_TIP_Half Guard Control Time  \
0                               0.0                                    0.0   
1                               0.0                                    0.0   
2                               0.0                                    0.0   
3                               0.0                                    0.0   
4                               0.0                                    0.0   

   R__Round5_TIP_Misc. Ground Control Time  R__Round5_TIP_Mount Control Time  \
0                                      0.0                               0.0   
1                                      0.0                               0.0   
2                                      0.0                               0.0   
3                                      0.0                               0.0   
4                                      0.0                               0.0   

   R__Round5_TIP_Neutral Time  R__Round5_TIP_Side Control Time  \
0                         0.0                              0.0   
1                         0.0                              0.0   
2                         0.0                              0.0   
3                         0.0                              0.0   
4                         0.0                              0.0   

   R__Round5_TIP_Standing Time   winby  winner  
0                          0.0     DEC     red  
1                          0.0     SUB    blue  
2                          0.0  KO/TKO     red  
3                          0.0     SUB    blue  
4                          0.0     DEC     red  

[5 rows x 895 columns]

In [16]:
idealcol = ['BPrev','BStreak','B_Age','B_Height','B_Weight','RPrev','RStreak','R_Age','R_Height','R_Weight']
ufc_zero[idealcol].head()

BPrev  BStreak  B_Age  B_Height  B_Weight  RPrev  RStreak  R_Age  R_Height  \
0      1        1   23.0     182.0        84      6        1   27.0       187   
1      0        0   32.0     175.0        70      0        0   29.0       182   
2      2        0   38.0     172.0        70      2        0   32.0       177   
3      0        0   23.0     170.0        56      6        4   25.0       175   
4      3        1   30.0     167.0        61      5        2   28.0       170   

   R_Weight  
0        84  
1        70  
2        70  
3        56  
4        61

In [17]:
ufc_zero["winner"]= ufc_zero["winner"].replace("red", 0)
ufc_zero["winner"]= ufc_zero["winner"].replace("blue", 1)
ufc_zero.head()

BPrev  BStreak  B_Age  B_Height                          B_HomeTown  B_ID  \
0      1        1   23.0     182.0                        Trento Italy  2783   
1      0        0   32.0     175.0  Careiro da Várzea, Amazonas Brazil  2208   
2      2        0   38.0     172.0                      Kanagawa Japan   721   
3      0        0   23.0     170.0                      Tijuana Mexico  2825   
4      3        1   30.0     167.0                     Spokane, WA USA  2260   

          B_Location                 B_Name  B_Weight  \
0  Mezzocorona Italy         Marvin Vettori        84   
1   Pharr, Texas USA  Carlos Diego Ferreira        70   
2        Tokyo Japan          Takanori Gomi        70   
3     Tijuana Mexico         Brandon Moreno        56   
4    Spokane, WA USA     Elizabeth Phillips        61   

   B__Round1_Grappling_Reversals_Landed  ...  R__Round5_TIP_Ground Time  \
0                                   0.0  ...                        0.0   
1                                   0.0  ...                        0.0   
2                                   0.0  ...                        0.0   
3                                   0.0  ...                        0.0   
4                                   0.0  ...                        0.0   

   R__Round5_TIP_Guard Control Time  R__Round5_TIP_Half Guard Control Time  \
0                               0.0                                    0.0   
1                               0.0                                    0.0   
2                               0.0                                    0.0   
3                               0.0                                    0.0   
4                               0.0                                    0.0   

   R__Round5_TIP_Misc. Ground Control Time  R__Round5_TIP_Mount Control Time  \
0                                      0.0                               0.0   
1                                      0.0                               0.0   
2                                      0.0                               0.0   
3                                      0.0                               0.0   
4                                      0.0                               0.0   

   R__Round5_TIP_Neutral Time  R__Round5_TIP_Side Control Time  \
0                         0.0                              0.0   
1                         0.0                              0.0   
2                         0.0                              0.0   
3                         0.0                              0.0   
4                         0.0                              0.0   

   R__Round5_TIP_Standing Time   winby  winner  
0                          0.0     DEC       0  
1                          0.0     SUB       1  
2                          0.0  KO/TKO       0  
3                          0.0     SUB       1  
4                          0.0     DEC       0  

[5 rows x 895 columns]

In [18]:
from sklearn.model_selection import train_test_split

feature_cols = ['BPrev','BStreak','B_Age','B_Height','B_Weight','RPrev','RStreak','R_Age','R_Height','R_Weight']
X = ufc_zero[feature_cols] # Features
y = ufc_zero.winner # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [19]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [20]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
from tensorflow.keras.utils import to_categorical

In [22]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [23]:
y_train_categorical

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [24]:
y_test_categorical

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



In [26]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_dim=10))
model.add(Dense(units=2, activation='softmax'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 88        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 18        
Total params: 106
Trainable params: 106
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100, # should be 100
    shuffle=True,
    verbose=2
)

Train on 1088 samples
Epoch 1/100
1088/1088 - 1s - loss: 0.8941 - accuracy: 0.5009
Epoch 2/100
1088/1088 - 0s - loss: 0.8342 - accuracy: 0.5230
Epoch 3/100
1088/1088 - 0s - loss: 0.7939 - accuracy: 0.5184
Epoch 4/100
1088/1088 - 0s - loss: 0.7626 - accuracy: 0.5239
Epoch 5/100
1088/1088 - 0s - loss: 0.7387 - accuracy: 0.5211
Epoch 6/100
1088/1088 - 0s - loss: 0.7194 - accuracy: 0.5414
Epoch 7/100
1088/1088 - 0s - loss: 0.7045 - accuracy: 0.5506
Epoch 8/100
1088/1088 - 0s - loss: 0.6933 - accuracy: 0.5542
Epoch 9/100
1088/1088 - 0s - loss: 0.6849 - accuracy: 0.5643
Epoch 10/100
1088/1088 - 0s - loss: 0.6786 - accuracy: 0.5800
Epoch 11/100
1088/1088 - 0s - loss: 0.6741 - accuracy: 0.5827
Epoch 12/100
1088/1088 - 0s - loss: 0.6702 - accuracy: 0.5919
Epoch 13/100
1088/1088 - 0s - loss: 0.6672 - accuracy: 0.5974
Epoch 14/100
1088/1088 - 0s - loss: 0.6645 - accuracy: 0.6020
Epoch 15/100
1088/1088 - 0s - loss: 0.6628 - accuracy: 0.6002
Epoch 16/100
1088/1088 - 0s - loss: 0.6612 - accuracy: 0.

In [35]:
#Saving the tf.keras model
model.save('tf_model3')

INFO:tensorflow:Assets written to: tf_model3/assets


In [ ]:
#Loading the model back
from tensorflow import keras
model = keras.models.load_model("tf_model3")

In [ ]:
deep_model = Sequential()
deep_model.add(Dense(units=8, activation='relu', input_dim=10))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [ ]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
ynew = model.predict(X_test_scaled)

In [ ]:
ynew

In [ ]:
y

In [ ]:
y_test.head()
y_test.describe

In [ ]:
ynew_new = np.round(ynew)

In [ ]:
for i in range(len(X_test_scaled)):
    if np.round(ynew[i][0]) == 1:
        pred_win = 'blue'
    else:
        pred_win = 'red'
        
    print("X=%s, Predicted=%s" % (X_test_scaled[i], pred_win))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, ynew))